**File explorer** to find the file you want to analyse with **Statement Processor** to generate all data and visualisations

In [ ]:
%run ProcessxAPISGStatement.ipynb # notebook to process an xAPI-SG statement
%run widgets/selectFileWidget.ipynb
%run widgets/simvaWidget.ipynb
global location_file, filename,progressbarFile,progressbarTraces, checkboxesPlayersSelected
progressbarFile = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
progressbarTraces = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
location_file=""
filename=""

if storage == 'simva' :
    global simvaWidget
    #*** THE MAIN NOTEBOOK MUST HAVE A a variable of type ipyauth.Auth ***
    simvaWidget = SimvaBrowser(auth=a, storage_url=simva_storage_url, ca_file=simva_ca_file, accept='.json')

    def on_file_load(change):
        global location_file, checkboxesPlayersSelected, filename
        players_info.clear()
        # file with xAPI-SG statements
        location_file=simvaWidget.current_path
        basename=location_file.rpartition(simvaWidget.delimiter)[2]
        info = basename.partition('.')
        filename = info[0]
        try:
            start_time = datetime.now()
            ## FILE MUST CONTAIN LIST OF XAPI-SG PROFILE TRACES
            ## (traces separated by commas and enclosed by [])
            statements_str = simvaWidget.get_file_content(location_file)
            nblines=len(statements_str.splitlines())
            trace=0
            for s in statements_str.splitlines():
                progressbarTraces.value=trace/nblines
                trace=trace+1
                statement = json.loads(s)
                process_xapisg_statement(statement, players_info, timeformats)
            end_time = datetime.now()
        except Exception as e:
            with simvaWidget.output:
                print("FILE MUST CONTAIN A LIST XAPI-SG PROFILE TRACES (one trace by line)")
                print("This file cannot be analyse by the processor per statement xAPI. Please select an another file.")
                clear_output(wait=True)
        progressbarTraces.value = 1.0
        with simvaWidget.output:
            print("Process per statement OK")
            print("Displaying all vis...")
            display_checkboxes()
            clear_output(wait=True)
        displayvis(None)
    simvaWidget.buttonRun.on_click(on_file_load)
else:
    if local:
        global fileBrowser
        #*** !TO BE USED WHEN THE NOTEBOOK IS RUNNED LOCALLY ***
        fileBrowser = FileBrowser(accept='.json')

        def on_file_load(change):
            global location_file, checkboxesPlayersSelected, filename
            players_info.clear()
            # file with xAPI-SG statements
            location_file=fileBrowser.path
            basename=os.path.basename(location_file)
            info = os.path.splitext(basename)
            filename = info[0]
            try:
                start_time = datetime.now()
                ## FILE MUST CONTAIN LIST OF XAPI-SG PROFILE TRACES
                ## (traces separated by commas and enclosed by [])
                trace=0
                with open(location_file, 'r', encoding='UTF-8')  as file:
                    nblines=len(f.splitlines())
                    while line := file.readline():
                        progressbarTraces.value=trace/nblines
                        trace=trace+1
                        statement = json.load(line.rstrip())
                        process_xapisg_statement(statement, players_info, timeformats)
                    end_time = datetime.now()
            except Exception as e:
                with fileBrowser.output:
                    print("FILE MUST CONTAIN A LIST XAPI-SG PROFILE TRACES (one trace by line)")
                    print("This file cannot be analyse by the processor per statement xAPI. Please select an another file.")
                    clear_output(wait=True)
            progressbarTraces.value = 1.0
            with fileBrowser.output:
                print("Process per statement OK")
                print("Displaying all vis...")
                display_checkboxes()
                clear_output(wait=True)
            displayvis(None)
        fileBrowser.buttonRun.on_click(on_file_load)

    else:
        global upload_button
        #***  !TO BE USED WHEN THE NOTEBOOK IS HOSTED ON THE WEB ***
        upload_button=widgets.FileUpload(
            description='Upload xAPI data',
            accept='.json',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
            multiple=True,  # True to accept multiple files upload, else False
            layout=Layout(width='25%')
        )
        global outTabs,ErrorOut, uploadButtonApp
        outTabs=widgets.Output()
        ErrorOut=widgets.Output()
        uploadButtonApp=VBox([upload_button, progressbarFile, progressbarTraces, ErrorOut, outTabs])

        #Observe the file load widget (online version)
        def on_file_upload(change):
            outTabs.clear_output()
            ErrorOut.clear_output()
            upload_button._counter=0
            progressbarFile.value=0.0
            progressbarTraces.value=0.0
            filename=""
            number_of_files = len(upload_button.value) # number of json files selected
            players_info.clear()
            errors={}
            for file_index in range(0,number_of_files):
                progressbarFile.value = file_index/number_of_files
                location_file = upload_button.value[file_index].name #get the file name from the embedded metadata dict
                file = '-'.join(location_file.split(".")[:-1])
                f = upload_button.value[file_index].content.tobytes().decode("utf-8") #extract data byte string and convert to str
                filename += file + "-"
                try:
                    nblines = len(f.splitlines())
                    start_time = datetime.now()
                    ## FILE MUST CONTAIN LIST OF XAPI-SG PROFILE TRACES
                    ## (traces separated by commas and enclosed by [])
                    trace=0
                    for s in f.splitlines() :
                        progressbarTraces.value=trace/nblines
                        trace=trace+1
                        try:
                            statement = json.loads(s)
                            process_xapisg_statement(statement, players_info, timeformats)
                        except Exception as e:
                            errors.append(s)
                    end_time = datetime.now()
                    progressbarFile.value = 1.0
                    progressbarTraces.value = 1.0
                except Exception as e:
                    with ErrorOut:
                        print("FILE MUST CONTAIN A LIST XAPI-SG PROFILE TRACES (one trace by line)")
                        print("This file" + location_file + "cannot be analyse by the processor per statement xAPI. Please select an another file.")
                        players_info.clear()
            with ErrorOut:
                nbError=len(errors)
                print("Nb error : ", len(errors), " | Nb total lines:", nblines)
                if nbError == nblines: 
                    print("FILE MUST CONTAIN A LIST XAPI-SG PROFILE TRACES (one trace by line)")
                    print("This file" + location_file + "cannot be analyse by the processor per statement xAPI. Please select an another file.")
                    players_info.clear()
            with outTabs:
                    print("Process per statement OK")
                    print("Displaying all vis...")
                    clear_output(wait=True)
                    display_checkboxes()
            displayvis(None)
            upload_button._counter=0
        upload_button.observe(on_file_upload, names='value')

def displayvis(change):
    if storage == 'simva' :
        with simvaWidget.output:
            displayAllVisualisations()
        simvaWidget.output.clear_output(wait=True)
    else:
        if local:
            with fileBrowser.output:
                displayAllVisualisations()
            fileBrowser.output.clear_output(wait=True)
        else:
            if not(players_info == {}) :
                #ErrorOut.clear_output()
                with outTabs:
                    displayAllVisualisations()
                outTabs.clear_output(wait=True)